In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.utils import to_categorical

C:\Users\llreb\Anaconda3\envs\PythonData\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# lenders
USB= 504713         # US Bank
Wells=451965        #Wells Fargo
Quicken=7197000003  #Quicken Loan

loans_raw = pd.read_csv('AllData2017.csv')
loans_df = loans_raw.loc[(loans_raw['respondent id'] == Wells) | (loans_raw['respondent id'] == USB) | (loans_raw['respondent id'] == Quicken), :]
#loans_df = loans_raw.loc[(loans_raw['respondent id'] == Wells), :]
#loans_df = loans_raw.loc[(loans_raw['respondent id'] == USB), :]
#loans_df = loans_raw.loc[(loans_raw['respondent id'] == Quicken), :]
loans_df.head()

,Index,respondent id,agency code,loan type,property type,loan purpose,owner occ,loan amount,pre approval,action taken,...,denial reason 1,denial reason 2,denial reason 3,rate of spread,hoepa status,lien status,loan type modified,action modified,income cleaned,income loan ratio
0,1,451965,9,1,1,3,1,43,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,41,105
1,2,451965,9,2,1,1,1,53,3,1,...,NaN,NaN,NaN,NaN,2,1,0,1,0,0
2,3,451965,9,1,1,3,1,114,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,55,207
3,4,451965,9,1,1,3,1,142,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,75,189
4,5,451965,9,1,1,1,1,162,3,1,...,NaN,NaN,NaN,NaN,2,1,1,1,0,0


In [3]:
# Assign X (data) and y (target)
#drop all of the columns with non-numeric data
X=loans_df.drop(["action modified","respondent id", "state", "county", "loan type", "owner occ", "census tract", "hoepa status", "Index", "agency code", "income", "msa/md", "pre approval", "action taken", "denial reason 1", "denial reason 2", "rate of spread", "denial reason 3", "type of purchaser"],axis=1)

y=loans_df["action modified"]
print(X.shape, y.shape)
X.head()

(34071, 9) (34071,)


,property type,loan purpose,loan amount,applicant race 1,sex,lien status,loan type modified,income cleaned,income loan ratio
0,1,3,43,5,2,1,1,41,105
1,1,1,53,5,2,1,0,0,0
2,1,3,114,6,3,1,1,55,207
3,1,3,142,3,1,1,1,75,189
4,1,1,162,2,2,1,1,0,0


In [5]:
X_continuous = pd.DataFrame(X, columns=['income cleaned', 'loan amount', 'income loan ratio'])

# Create a StandardScater model and fit it to the training data
X_scaler = MinMaxScaler().fit(X_continuous)


In [6]:
# Transform the training and testing data using the X_scaler

X_scaled = X_scaler.transform(X_continuous)

In [7]:
# One-hot encoding
property_type = to_categorical(X["property type"])
loan_purpose = to_categorical(X["loan purpose"])
race = to_categorical(X["applicant race 1"])
sex = to_categorical(X["sex"])
lien_status = to_categorical(X["lien status"])
loan_type = to_categorical(X["loan type modified"])


In [8]:
categorical_X = np.hstack([property_type, loan_purpose, race, sex, lien_status, loan_type])

print(categorical_X.shape)


(34071, 26)


In [10]:
all_X = np.hstack([categorical_X, X_scaled])

print(all_X[0])


[0.         1.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         0.
 1.         0.         0.         0.         0.         1.
 0.         0.         0.         1.         0.         0.
 0.         1.         0.00594203 0.00781395 0.0153666 ]


In [11]:
# Use train_test_split to create training and testing data

X_train, X_test, y_train, y_test = train_test_split(all_X, y)
#X_train.dtypes

In [12]:
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [14]:
# First, create a normal neural network with 
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=29))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

In [15]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# Fit the model to the training data
model.fit(
    X_train,
    y_train_categorical,
    epochs=500,
    shuffle=True,
    verbose=2
)

Epoch 1/500
 - 1s - loss: 0.5001 - acc: 0.7739
Epoch 2/500
 - 1s - loss: 0.4946 - acc: 0.7748
Epoch 3/500
 - 1s - loss: 0.4939 - acc: 0.7746
Epoch 4/500
 - 1s - loss: 0.4917 - acc: 0.7770
Epoch 5/500
 - 1s - loss: 0.4907 - acc: 0.7772
Epoch 6/500
 - 1s - loss: 0.4896 - acc: 0.7793
Epoch 7/500
 - 1s - loss: 0.4874 - acc: 0.7807
Epoch 8/500
 - 1s - loss: 0.4867 - acc: 0.7823
Epoch 9/500
 - 1s - loss: 0.4861 - acc: 0.7819
Epoch 10/500
 - 1s - loss: 0.4865 - acc: 0.7816
Epoch 11/500
 - 1s - loss: 0.4854 - acc: 0.7825
Epoch 12/500
 - 1s - loss: 0.4851 - acc: 0.7832
Epoch 13/500
 - 1s - loss: 0.4850 - acc: 0.7824
Epoch 14/500
 - 1s - loss: 0.4850 - acc: 0.7822
Epoch 15/500
 - 1s - loss: 0.4842 - acc: 0.7828
Epoch 16/500
 - 1s - loss: 0.4834 - acc: 0.7839
Epoch 17/500
 - 1s - loss: 0.4834 - acc: 0.7832
Epoch 18/500
 - 1s - loss: 0.4831 - acc: 0.7848
Epoch 19/500
 - 1s - loss: 0.4824 - acc: 0.7846
Epoch 20/500
 - 1s - loss: 0.4822 - acc: 0.7842
Epoch 21/500
 - 1s - loss: 0.4827 - acc: 0.7835
E

Epoch 171/500
 - 1s - loss: 0.4631 - acc: 0.7943
Epoch 172/500
 - 1s - loss: 0.4625 - acc: 0.7940
Epoch 173/500
 - 1s - loss: 0.4640 - acc: 0.7935
Epoch 174/500
 - 1s - loss: 0.4621 - acc: 0.7936
Epoch 175/500
 - 1s - loss: 0.4621 - acc: 0.7934
Epoch 176/500
 - 1s - loss: 0.4616 - acc: 0.7947
Epoch 177/500
 - 1s - loss: 0.4623 - acc: 0.7939
Epoch 178/500
 - 1s - loss: 0.4616 - acc: 0.7949
Epoch 179/500
 - 1s - loss: 0.4617 - acc: 0.7938
Epoch 180/500
 - 1s - loss: 0.4616 - acc: 0.7940
Epoch 181/500
 - 1s - loss: 0.4616 - acc: 0.7944
Epoch 182/500
 - 1s - loss: 0.4625 - acc: 0.7938
Epoch 183/500
 - 1s - loss: 0.4613 - acc: 0.7940
Epoch 184/500
 - 1s - loss: 0.4612 - acc: 0.7949
Epoch 185/500
 - 1s - loss: 0.4608 - acc: 0.7950
Epoch 186/500
 - 1s - loss: 0.4608 - acc: 0.7945
Epoch 187/500
 - 1s - loss: 0.4606 - acc: 0.7949
Epoch 188/500
 - 1s - loss: 0.4619 - acc: 0.7946
Epoch 189/500
 - 1s - loss: 0.4614 - acc: 0.7951
Epoch 190/500
 - 1s - loss: 0.4605 - acc: 0.7956
Epoch 191/500
 - 1s 

 - 1s - loss: 0.4512 - acc: 0.7994
Epoch 339/500
 - 1s - loss: 0.4504 - acc: 0.8004
Epoch 340/500
 - 1s - loss: 0.4499 - acc: 0.8007
Epoch 341/500
 - 1s - loss: 0.4519 - acc: 0.8011
Epoch 342/500
 - 1s - loss: 0.4507 - acc: 0.8013
Epoch 343/500
 - 1s - loss: 0.4502 - acc: 0.8005
Epoch 344/500
 - 1s - loss: 0.4501 - acc: 0.8011
Epoch 345/500
 - 1s - loss: 0.4500 - acc: 0.8016
Epoch 346/500
 - 1s - loss: 0.4514 - acc: 0.8003
Epoch 347/500
 - 1s - loss: 0.4510 - acc: 0.8008
Epoch 348/500
 - 1s - loss: 0.4497 - acc: 0.8002
Epoch 349/500
 - 1s - loss: 0.4499 - acc: 0.8018
Epoch 350/500
 - 1s - loss: 0.4508 - acc: 0.7996
Epoch 351/500
 - 1s - loss: 0.4494 - acc: 0.8009
Epoch 352/500
 - 1s - loss: 0.4487 - acc: 0.8008
Epoch 353/500
 - 1s - loss: 0.4480 - acc: 0.8009
Epoch 354/500
 - 1s - loss: 0.4504 - acc: 0.8005
Epoch 355/500
 - 1s - loss: 0.4502 - acc: 0.8012
Epoch 356/500
 - 1s - loss: 0.4503 - acc: 0.8004
Epoch 357/500
 - 1s - loss: 0.4489 - acc: 0.8012
Epoch 358/500
 - 1s - loss: 0.4515

In [19]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(
    f"Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Neural Network - Loss: 0.5664471569238176, Accuracy: 0.7828128668561624


In [21]:
#make a prediction and compare it to the actual labels
predictions = model.predict_classes(X_test)
predict_df = pd.DataFrame({"Prediction": predictions, "Actual": y_test})

In [23]:
# Calculate confusion matrix
from sklearn.metrics import confusion_matrix
output = confusion_matrix(y_test, predictions)
output

array([[ 253, 1642],
       [ 208, 6415]], dtype=int64)

In [ ]:
output_df = pd.DataFrame(output)
output_df.to_csv("confusion_matrix.csv")